### Demo Preparation Notebook

**Please Note**: This notebook and demo are NOT intended to be used as learning materials. To gain
a thorough understanding of the DataJoint workflow for calcium imaging, please
see the [`tutorial`](./tutorial.ipynb) notebook.

In [ ]:
import os

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

import datajoint as dj
import datetime
import numpy as np
from workflow_calcium_imaging.pipeline import subject, session, scan, imaging
from element_calcium_imaging import imaging_report
import suite2p

In [ ]:
subject.Subject.insert1(
    dict(
        subject='subject1',
        subject_birth_date='2023-01-01',
        sex='U',
    )
)

In [ ]:
Equipment.insert1(dict(scanner="Mesoscope"))

In [ ]:
session_key = dict(subject='subject1', 
                   session_datetime=datetime.datetime.now())

session.Session.insert1(session_key)

session.SessionDirectory.insert1(
    dict(
        session_key, 
        session_dir='subject1/session1'
    )
)

In [ ]:
scan.Scan.insert1(
    dict(
        session_key,
        scan_id=0,
        scanner="Mesoscope",
        acq_software='ScanImage',
    )
)

In [ ]:
populate_settings = {"display_progress": True}

scan.ScanInfo.populate(**populate_settings)

In [ ]:
suite2p_params = suite2p.default_ops()
suite2p_params['nonrigid']=False

imaging.ProcessingParamSet.insert_new_params(
    processing_method="suite2p",
    paramset_idx=0,
    params=suite2p_params,
    paramset_desc='Default parameter set for suite2p'
)

In [ ]:
imaging.ProcessingTask.insert1(
    dict(
        session_key,
        scan_id=0,
        paramset_idx=0,
        task_mode='load', # load or trigger
        processing_output_dir='subject1/session1/suite2p',
    )
)

imaging.Processing.populate(**populate_settings)

In [ ]:
imaging.Curation.insert1(
    dict(
        session_key,
        scan_id=0,
        paramset_idx=0,
        curation_id=0,
        curation_time=datetime.datetime.now(),
        curation_output_dir='subject1/session1/suite2p',
        manual_curation=False,
    )
)

imaging.MotionCorrection.populate(**populate_settings)
imaging.Segmentation.populate(**populate_settings)
imaging.Fluorescence.populate(**populate_settings)
imaging.Activity.populate(**populate_settings)
imaging_report.ScanLevelReport.populate()
imaging_report.TraceReport.populate()